# Basic concepts

### Entity path

```text
"world/points"
"camera/image"
"camera/rays"
```

Like ROS topics, but hierarchical.

# Time

You explicitly set time:

```python
rr.set_time("video_frame", sequence=frame_idx)
```

or

```cpp
rr::set_time("video_frame", frame_idx);
```

Without time, everything is static.

---

# Logging Images


```python
import rerun as rr
import numpy as np
import cv2

rr.init("image_demo", spawn=True)

img = cv2.imread("image.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

rr.log("camera/image", rr.Image(img))
```

---


```cpp
#include <rerun.hpp>
#include <opencv2/opencv.hpp>

cv::Mat img = cv::imread("image.png");
cv::cvtColor(img, img, cv::COLOR_BGR2RGB);

rerun::log(
    "camera/image",
    rerun::Image(img.data, {img.cols, img.rows}, rerun::ColorModel::RGB)
);
```

---

#  Video (image sequence over time)

### Key idea:

**Video = images + time**


```python
cap = cv2.VideoCapture("video.mp4")

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    rr.set_time("frame", frame_idx)
    rr.log("camera/image", rr.Image(frame))

    frame_idx += 1
```

---


```cpp
cv::VideoCapture cap("video.mp4");
int frame_idx = 0;

cv::Mat frame;
while (cap.read(frame)) {
    cv::cvtColor(frame, frame, cv::COLOR_BGR2RGB);

    rerun::set_time_sequence("frame", frame_idx);

    rerun::log(
        "camera/image",
        rerun::Image(frame.data, {frame.cols, frame.rows})
    );

    frame_idx++;
}
```

---

#  2D & 3D Points

## 2D keypoints

```python
pts = np.array([[100, 50], [120, 80], [200, 90]])

rr.log(
    "image/keypoints",
    rr.Points2D(pts, radii=3)
)
```

## 3D point cloud

```python
points = np.random.randn(1000, 3)

rr.log(
    "world/points",
    rr.Points3D(points)
)
```

---

```cpp
std::vector<rerun::Vec3D> points;
for (int i = 0; i < 1000; ++i) {
    points.emplace_back(randf(), randf(), randf());
}

rerun::log("world/points", rerun::Points3D(points));
```

---

#  Shapes (Lines, Boxes, Arrows)

## Line strip (trajectory)



```python
trajectory = np.cumsum(np.random.randn(100, 3) * 0.1, axis=0)

rr.log(
    "world/trajectory",
    rr.LineStrips3D([trajectory])
)
```

---


```cpp
std::vector<rerun::Vec3D> traj;
...
rerun::log(
    "world/trajectory",
    rerun::LineStrips3D({traj})
);
```

---

## Boxes

```python
rr.log(
    "world/box",
    rr.Boxes3D(
        centers=[[0,0,0]],
        sizes=[[1,1,1]]
    )
)
```

---

#  Camera

Rerun treats cameras **explicitly**

---

## Camera intrinsics


```python
rr.log(
    "world/camera",
    rr.Pinhole(
        resolution=[640, 480],
        focal_length=[500, 500],
        principal_point=[320, 240]
    )
)
```

---


```cpp
rerun::log(
    "world/camera",
    rerun::Pinhole(
        {640, 480},
        {500.0f, 500.0f},
        {320.0f, 240.0f}
    )
);
```

---

## Camera pose (extrinsics)

Camera pose = transform in world.



```python
rr.log(
    "world/camera",
    rr.Transform3D(
        translation=[0, 0, 1],
        rotation=rr.Quaternion(xyzw=[0,0,0,1])
    )
)
```

---

```cpp
rerun::log(
    "world/camera",
    rerun::Transform3D(
        rerun::Vec3D{0,0,1},
        rerun::Quaternion{0,0,0,1}
    )
);
```

---

# Camera rays 

* **hovering pixel → ray**
* requires:

  * image
  * pinhole
  * transform

If you log **image + pinhole + transform** under `"world/camera"`, Rerun automatically enables:

* pixel hover
* ray visualization
* projection into 3D
---


```python
import rerun as rr
import time
import numpy as np
import cv2

rr.init("image_demo", spawn=True)

rr.log("world", rr.ViewCoordinates.RIGHT_HAND_Z_UP, static=True)

rr.log(
    "world/axes",
    rr.Arrows3D(
        vectors=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
        colors=[[255, 0, 0], [0, 255, 0], [0, 0, 255]],
    ),
)


rr.log(
    "world/camera",
    rr.Pinhole(
        resolution=[640, 480],
        focal_length=[500, 500],
        principal_point=[320, 240]
    )
)

rr.log(
    "world/camera",
    rr.Transform3D(
        translation=[0, 0, 1],
        rotation=rr.Quaternion(xyzw=[0, 0, 0, 1])
    )
)


img = cv2.imread("iamge.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

rr.log("world/camera", rr.Image(img))

```

---

# Rerun View Coordinates

`rr.ViewCoordinates` is an archetype you log to define the **semantic meaning** of the X, Y, and Z axes.
This tells the Rerun Viewer how to interpret your coordinate system, which affects.

It does **not** transform or flip your actual point positions — it only influences how the viewer orients the scene for better usability.

### Specific Constants

**Rerun** Default
- **`rr.ViewCoordinates.RIGHT_HAND_Z_UP`**  
  Defines the axes as: **X = Right**, **Y = Forward**, **Z = Up**



**OpenCV** (RDF: Right-Down-Forward) convention:
- **`rr.ViewCoordinates.RIGHT_HAND_Y_DOWN`**  
  Defines the axes as: **X = Right**, **Y = Down**, **Z = Forward**  
  
```python
rr.log("world", rr.ViewCoordinates.RIGHT_HAND_Y_DOWN, static=True)

rr.log("world/points",
       rr.Points3D([[0, 0, 0], [1, 0, 0], [0, 0, 4]], radii=0.2))


rr.log(
    "world/axes",
    rr.Arrows3D(
        vectors=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
        colors=[[255, 0, 0], [0, 255, 0], [0, 0, 255]],
    ),
)

```
<img src="images/rerun_RDF_opencv_coordinate.png" />



# OpenCV Images in Rerun


Rerun world:

```
X → right
Y → forward
Z → up
```

OpenCV:

```
X → right
Y → down
Z → forward
```

### Position

```
X_rerun =  X_cv
Y_rerun =  Z_cv
Z_rerun = -Y_cv
```

### Rotation

$$
R_{\text{rerun}\leftarrow\text{cv}} =
\begin{bmatrix}
1 & 0 & 0  \\
0 & 0 & 1  \\
0 & -1 & 0
\end{bmatrix}
$$

For integrating OpenCV camera data with Rerun when you want a **Z-up world** (common in robotics, AR, etc.).

```python
points_cv = np.array([
    [0.0,  0.0, 2.0],   # straight ahead
    [0.5,  0.0, 2.0],   # right
    [-0.5, 0.0, 2.0],   # left
    [0.0,  0.5, 2.0],   # down
])

# ============================================================
# Convert OpenCV → Rerun world
# World: X=right, Y=forward, Z=up
# ============================================================
points_world = np.column_stack([
    points_cv[:, 0],    # X stays X
    points_cv[:, 2],    # Z (forward) → Y
    -points_cv[:, 1],   # -Y (down) → Z (up)
])

```



### Should you instead "just choose Z down" (e.g., make world Y-down)?
No — only if you specifically want a Y-down world (e.g., matching some simulations or datasets). But:
- It would require **no point transformation** (you could log points_cv directly).
- You'd change the world to something like `rr.ViewCoordinates.RIGHT_HAND_Y_DOWN` (X right, Y down, Z forward).
- The camera pose could be identity (camera forward = +Z_world).
- However, you'd lose the natural "Z up" viewing experience in Rerun's 3D viewer for ground-plane robotics/AR scenarios.

Your current approach (transform points + pose, keep Z-up world) is more common and flexible when working with real-world upward gravity.

# How to Animate Objects in Rerun

Animating objects (like a camera, robot, or points) in a Rerun scene is simple and powerful thanks to its timeline-based logging.

The key idea: **log the same entity path at different times** with updated transforms or data.


1. **Set a timeline** for each frame:  
   Use `rr.set_time("frame", frame_idx)`  before logging data in a loop.

2. **Log time-varying data** to the same entity path:  
   For a moving object (e.g., a camera), repeatedly log a `rr.Transform3D` with new translation/rotation:
   ```python
   rr.log("world/camera", rr.Transform3D(
       translation=[x, y, z],
       rotation=rr.Quaternion(xyzw=[qx, qy, qz, qw])
   ))
   ```

3. **Log associated data** (images, points, etc.) at the same timestamp:  
   They will automatically synchronize with the transform.


```python
import rerun.blueprint as rrb
import rerun as rr
import cv2
import numpy as np
import time
from pathlib import Path


rr.init("rerun_demo", spawn=True)
rr.log("world/xyz", rr.Arrows3D(vectors=[[1, 0, 0], [0, 1, 0],
       [0, 0, 1]], colors=[[255, 0, 0], [0, 255, 0], [0, 0, 255]],),)


# ============================================================================
# Setup Camera and Webcam
# ============================================================================
resolution = [1280, 720]
fx = 848.53117539872062
fy = 848.53117539872062
cx = 639.5
cy = 359.5

cap = cv2.VideoCapture(0)
# Set camera resolution to 1280x720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, resolution[0])
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, resolution[1])

# Open a video file (alternative)
# cap = cv2.VideoCapture("video.mp4")

entity_path = "world/camera"

# Log camera intrinsics (static - only once)
rr.log(
    entity_path,
    rr.Pinhole(
        resolution=[resolution[0], resolution[1]],
        focal_length=[fx, fy],
        principal_point=[cx, cy],
        # Distance from camera origin to image plane for 3D visualization
        image_plane_distance=1.0,
        color=[255, 128, 0],  # Orange color for camera frustum
        line_width=0.003  # Width of camera frustum lines
    ),
    static=True  # Camera intrinsics are static
)

# ============================================================================
# Live webcam feed with animated camera position
# ============================================================================
num_frames = 90  # Number of frames to capture

for frame_idx in range(num_frames):
    # Set time for this frame
    rr.set_time("frame", sequence=frame_idx)

    # Read webcam frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame from webcam")
        break

    # Convert BGR to RGB for Rerun
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Log image to camera (this will display in camera view)
    rr.log(entity_path, rr.Image(img_rgb))

    # Update camera position (animate in world)
    # Move camera along X-axis over time
    translation = [0.02 * frame_idx, 0, 1]
    rr.log(
        entity_path,
        rr.Transform3D(
            translation=translation,
            rotation=rr.Quaternion(xyzw=[0, 0, 0, 1])
        )
    )

    # Small delay to allow Rerun to process and control frame rate
    time.sleep(0.1)

# Release webcam
cap.release()
print(f"Captured {num_frames} frames")


```